In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.matplotlib.rcParams['savefig.dpi'] = 144
import seaborn

# Recommendation Engine, Session 2

In [2]:
import pandas as pd
import numpy as np
from sklearn import base
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import RidgeCV, LinearRegression, SGDRegressor, Ridge
from sklearn.decomposition import TruncatedSVD
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

/home/christian/.miniconda2/envs/datacourse/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/christian/.miniconda2/envs/datacourse/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
def parse_movie_line(l):
    id_, title, cats = l.strip().split('::')
    return {'id': int(id_), 'title': title, 'year': int(title.rsplit(' ')[-1][1:-1]), 
            'categories': cats.split('|')}

with open('ml-10M100K/movies.dat', 'r') as f:
    df = pd.DataFrame([parse_movie_line(l) for l in f]).set_index('id')

def parse_rating_line(l):
    uid, mid, rating, time = l.strip().split('::')
    return {'user_id': int(uid), 'movie_id': int(mid), 'rating': float(rating)}

with open('ml-10M100K/ratings.dat', 'r') as f:
    df_ratings = pd.DataFrame([parse_rating_line(l) for l in f])

## Regression of Ratings

In the previous session, we developed several ways to recommend movies to a user.  It was hard to tell how well these systems worked, since most users have not rated most movies.  We may return a list of recommendations, none of which has been reviewed by the user.  Is that a sign the user doesn't like those movies, or that they just haven't seen the yet.

An alternative approach is develop a regression model to learn the ratings.  That is, given a user $u$ and an item $i$, we wish to build a model to predict $r_{ui}$, the rating given by $u$ to $i$.  In evaluating the performance of the model, we will consider only user/items pairs where a rating exists.  Our predictions, $\hat r_{ui}$ can be used to determine which unrated items may be preferred by a user.

In order to test how well we can predict rating that haven't been made, we train on only a portion of the data and then evaulate its perfomance on the remainder of the data.  We will take care to use a **train-test split** so that we remove a holdout **testing set**, independent of our **training set**, on whch we evaluate the reuslts .  We will use the root mean-squared error (RMSE) to evaulate our model.  That is, we will compute
$$ \mbox{RMSE} = \sqrt{\frac{1}{n} \sum_{ui} \left(\hat r_{ui} - r_{ui}\right)^2} $$
where $r_{ui}$ are the ratings for $n$ $ui$-pairs in the test set.
  

For the purposes of having code execute quickly for demonstration, we will work with only a small portion of the users.

In [4]:
users = df_ratings['user_id'].unique()
np.random.seed(42)
user_sample = np.random.choice(users, 3000)
df_ratings_sample = df_ratings[df_ratings['user_id'].isin(user_sample)]
X_train, X_test, y_train, y_test = train_test_split(df_ratings_sample, df_ratings_sample['rating'])

## Baseline Model

We start with a baseline model that accounts for the fact that different users and different items will have different baseline scores.  This model has no interaction between the user and item:

$$ b_{ui} = \mu + b_u + b_i \ . $$

This may seem silly to worry about, but the baseline model is very important in practice.  The [winners of the Netflix Grand Prize](http://netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf) succeeded largely on the strength of their baseline model.  Alone, it performed nearly as well as the existing Netflix model.

That model included a number of time-dependent terms, but we will consider each of the terms to be constant.  As such $\mu$ will be the average overall rating, $b_u$ will be the average difference from the global mean of all ratings by user $u$, and $b_i$ will be the average difference from the global mean of all ratings of item $i$.  If we one-hot encode both the users and movies, we can find these coefficients with a linear regression.

In [5]:
class Dictizer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col):
        self.col = col
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.col].apply(lambda x: {x: 1})

In [6]:
user_dict = Pipeline([('dict', Dictizer('user_id')),
                      ('vect', DictVectorizer())])
movie_dict = Pipeline([('dict', Dictizer('movie_id')),
                       ('vect', DictVectorizer())])
union = FeatureUnion([('user_dict', user_dict),
                      ('movie_dict', movie_dict)])
lr_pipe = Pipeline([('union', union),
                    ('lr', Ridge(alpha=0))])

In [7]:
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('user_dict', Pipeline(steps=[('dict', Dictizer(col='user_id')), ('vect', DictVectorizer(dtype=<type 'numpy.float64'>, separator='=', sort=True,
        sparse=True))])), ('movie_dict', Pipeline(steps=[('dict', Dictizer(col='movie_id')...it_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))])

In [8]:
len(lr_pipe.named_steps['lr'].coef_)

11374

A small function will help us calculate RMSE scores.

In [9]:
def rmse(model, X, y):
    return np.sqrt(mean_squared_error(model.predict(X), y))

To calibrate our expectations, the RMSE of the mean model is the square root of the variance.

In [10]:
np.sqrt(y_test.var())

1.0661338326706293

Our baseline model does about 18% better.

In [11]:
rmse(lr_pipe, X_test, y_test)

0.8798803883612244

However, our model scores significantly better on the training set.

In [12]:
rmse(lr_pipe, X_train, y_train)

0.84778024105566674

## Overfitting and Cross-validation

This discrepancy is a sign that we may be **overfitting**.  Training data is never perfectly clean.  It contains both the underlying signal, which you want to model, and some random noise, which you do not.  If your model has enough flexibility, it may fit both the noise and the signal.  When tested against other data with the same signal but different noise, it will perform poorly.

The noise in this case is the same issue that prompted us to use a Bayesian mean to rate the movies suggested by neighbor users.  Prior to seeing any ratings for a item, for example, we would expect $b_i = 0$.  After seeing a single rating $r$, we shouldn't set $b_i = r - mu$.  Instead, we should update our estimate or $b_i$ somewhat in between these values, depending on our confidence in the prior.

In this case, we could add in prior ratings by hand.  A more general solution is to take advange of **regularized** linear models.  These models add a penalty for non-zero coefficients in the linear regression.  This reduces the flexibility of the model, thereby reducing the chance of overfitting.

We are using **ridge regression**, which adds a term proportion to $\beta^2$ to the quantity being minimized:

$$ \left|y - X\cdot\beta\right|^2 + \alpha\ \left|\beta\right|^2 \ . $$

Like linear regression, this model is analytically soluble.  (Ridge regression reduces to linear regression in the limit $\alpha\to0$.)  If the model is being used to take an average, as happens with one-hot encoding, the resultant coefficients will be

$$ \hat\beta_i = \frac{\sum_{j\mid X_{ij}=1} y_j}{n + \alpha} \ , $$

where $n$ is the number of $y_j$ included in the sum.  Note that this is equivalent to our previous Bayesian mean, with $\mu=0$ and $N=\alpha$.

The weight $\alpha$ is a **hyperparameter** of the model.  We must set it in order to ensure the best performance of the model.  To find the appropriate value, we use **cross-validation**: training the model on a traning set and evaluating its performance of a test set.  $\alpha$ is adjusted to achieve the best performance on the test set.

While we could use a static train-test split for the testing, there are several cross-validation methods that allow us to test performace across an entire data set, while still checking for overfitting.  The most common is ***k*-fold validation**.  In this scheme, the data set is split up into $k$ portions, or folds.  The model is trained on $k-1$ of them and tested on the last fold.  Then the folds are rotated and the process repeated.

Thus, for each candidate $\alpha$, we would need to train and test $k$ times and average the results.  This would become tedious quickly.  Luckily, *Scikit Learn* provides the `GridSearchCV` class that performs a **grid search** over our hyperparameters.  It takes an estimator as an argument, along with the grid of hyperparameters to be searched over.  (Note that parameters of elements of a pipe can be referenced with the `element-name__parameter-name` convention.) 

In [ ]:
gs = GridSearchCV(lr_pipe, {'lr__alpha': np.logspace(-1,1,5)}, n_jobs=-1)

`GridSearchCV` is itself an estimator.  When it is fit, it will run 3-fold cross validation (by default) on each set of hyperparameters, choose the best set of hyperparameters, and then train the estimator with the best hyperparameters on the full set of data.

In [ ]:
gs.fit(X_train, y_train)

We see that performance on the test set has improved somewhat.  (The predict method on the grid search object simply calls predict on the best fit estimator.)

In [ ]:
rmse(gs, X_test, y_test)

The best estimator, and its hyperparameter, are available as properties of the grid search object.  Scikit Learn uses the convention that a trailing underscore indicates a property available only after a model has been fit.

In [ ]:
best_lr = gs.best_estimator_
gs.best_params_

## Modeling Interaction

With a baseline model for each user and item, we now look to build a model that accounts for the residual left over.  That is, we wish to model

$$ d_{ui} \equiv r_{ui} - b_{ui} \ . $$

The current state of the art for interaction modeling is **matrix factorization**.  Taking a cue from our previous work with dimensional reduction, we suppose that there are a small number of factors $f$ that users like about movies.  Thus, the interaction can be understood by figuring out the mapping of movies to those factors and those factors to user preferences.  In other words, we assume we can write

$$ d_{ui} = \sum_f U_{uf} M_{fi} \ , $$

where $U$ records what factors each user prefers and $M$ records which factors are in each movie.  As with linear regression, the goal is to minimize the sum of squared errors

$$ \sum_{u,i} \left(d_{ui} - \sum_f U_{uf} M_{fi} \right)^2 \ . $$

The values in both $U$ and $M$ are unknown.  We can work to estimate them through a procedure called **alternating least squares**.  If we assume that $M$ is known, we can find $U$ through a least squares optimization much like that of linear regression:

$$ U = d\ M^T (M\ M^T)^{-1} \ . $$

Then, treating $U$ as known, we can solve for $M$:

$$ M = (U^T\ U)^{-1} U^T\ d \ . $$

Iterating this process should take us closer to the optimal solution.

To avoid the potential of overfitting, we add a regularization parameter $\alpha$ to the algorithm.  The quantity to be minimized becomes

$$ \sum_{u,i} \left(d_{ui} - \sum_f U_{uf} M_{fi} \right)^2 + \alpha \left( \sum_{u,f} U_{uf}^2 + \sum_{f,i} M_{fi}^2 \right) \ . $$

However, most of the ratings, and therefore the residuals $d_{ui}$, are unknown.  Thus, we restrict the sum to include only those pairs of $u,i$ for which $r_{ui}$ is defined.

This algorithm is implemented in the `ResidualMatixFactorization` class.  It takes a baseline estimator, from which the residuals are calculated, and three hyperparameters:
- `n_factors` is the number of underlying factors.
- `n_iters` is the number of iterations of the alternating least squares algorithm to run.
- `alpha` is the regularization parameter.

In [ ]:
# Algorithm adapted from http://bugra.github.io/work/notes/2014-04-19/alternating-least-squares-method-for-collaborative-filtering/
# Also note, there is (slow) movement to integrate something into Scikit-Learn 
# https://github.com/scikit-learn/scikit-learn/issues/6142

class ResidualMatrixFactorization(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self, base_est, n_factors, n_iters, alpha):
        self.base_est = base_est
        self.n_factors = n_factors
        self.n_iters = n_iters
        self.alpha = alpha
        
        self.user_index = self.rating_index = self.Q = self.W = self.user_f = self.movie_f = None
    
    def fit_step(self):
        if self.user_index is None:
            self.init_fit(X, y)
        
        alphas = self.alpha * np.eye(self.n_factors)
        for u, Wu in enumerate(self.W):
            movie_f = self.movie_f[:,Wu]
            self.user_f[u] = np.linalg.solve(
                np.dot(movie_f, movie_f.T) + alphas,
                np.dot(movie_f, self.Q[u, Wu])).T
        for i, Wi in enumerate(self.W.T):
            user_f = self.user_f[Wi,:]
            self.movie_f[:, i] = np.linalg.solve(
                np.dot(user_f.T, user_f) + alphas,
                np.dot(user_f.T, self.Q[Wi, i]))
        
        return self
    
    def fit(self, X, y):
        self.base_est.fit(X, y)
        residuals = y - self.base_est.predict(X)
        df = pd.DataFrame({'user_id': X['user_id'], 'movie_id': X['movie_id'], 'residuals': residuals})
        
        rating_mat = df.pivot_table('residuals', 'user_id', 'movie_id')
        self.user_index = rating_mat.index
        self.movie_index = rating_mat.columns
        
        self.Q = rating_mat.fillna(0).values
        self.W = (~rating_mat.isnull()).values
        self.user_f = np.random.rand(self.Q.shape[0], self.n_factors)
        self.movie_f = np.random.rand(self.n_factors, self.Q.shape[1])

        for n in xrange(self.n_iters):
            self.fit_step()
        
        return self
    
    def interaction_prediction(self, row):
        uid = row['user_id']
        mid = row['movie_id']
        if uid in self.user_index and mid in self.movie_index:
            return np.dot(self.user_f[self.user_index.get_loc(uid), :],
                          self.movie_f[:, self.movie_index.get_loc(mid)])
        return 0
        
    def predict(self, X):
        return self.base_est.predict(X) + X.apply(self.interaction_prediction, axis=1).values

We need to set the regularization parameter through cross validation.

In [ ]:
gs = GridSearchCV(ResidualMatrixFactorization(best_lr, 20, 10, 1),
                  {'alpha': [1,3,10,30,100]})
gs.fit(X_train, y_train)
gs.best_params_

It looks like we gain about several percent on our baseline model.  This may not seem like much, but the Netflix Grand Prize winners managed less than a 10% improvement on their baseline model.

In [ ]:
rmse(gs, X_test, y_test)

The number of factors can also be explored through cross-validation.  As this parameter slows the model significantly, we must weigh the execution time of the model along with its RMSE score.  Properly speaking, we should be exploring both hyperparameters at the same time.  However, this would increase the number of combinations to be tested.  Optimizing hyperparameters independently often yeilds values near the global optimum.

In [ ]:
gs = GridSearchCV(ResidualMatrixFactorization(best, 20, 10, 30),
                  {'n_factors': [10, 20, 50, 100]})
gs.fit(X_train, y_train)
gs.best_params_

A larger number of factors is better, but the improvement is marginal.  We'll stick with $n=20$ for performance.

In [ ]:
rmse(gs, X_test, y_test)

We have been using 10 iterations, somewhat arbitrarily.  Let's see how the errors are changing each step.

In [ ]:
rest = ResidualMatrixFactorization(best_lr, 20, 0, 30)
rest.fit(X_train, y_train)
errors = []
for i in xrange(10):
    rest.fit_step()
    errors.append((rmse(rest, X_test, y_test), rmse(rest, X_train, y_train)))
plt.plot(errors)
plt.legend(['Out-of-sample error', 'In-sample error'])

We're probably getting most of the available performance in those first 10 iterations.

Finally, let's see what we would recommend to user 9689 based on these ratings.

In [ ]:
predict_df = pd.DataFrame({'movie_id': X_train['movie_id'].unique(), 'user_id': 9689})
predict_df = predict_df.merge(df[['title']], left_on='movie_id', right_index=True)
predict_df['scores'] = rest.predict(predict_df)
predict_df.sort_values('scores', ascending=False).head(20)

##  Exercises

1. Implement another user-item interaction model.  Here are a few ideas:
   - Measure how similar the user is to fans of the item, and regress against that distance.
   - Measure how similar the item is to items the user likes, and regress against that distance.
   - Use a nearest neighbors approach to report the scores given to an item by users similar to the user in question.
   
2. Blend two (or more) interaction models together.  These can be different algorithms from (1) or several matrix factorization algorithms with different hyperparameters.  Linear regression is a good way to automatically perform a weighted average together.  The BellKor team had success with **gradient-boosted trees** as blenders.

3. Add features of the movie or user to the blend.  A nonlinear model, such as **random forest** or gradient-boosted trees, may be able to detect that one of the component models does particularly well for a certain subset of features.